<a href="https://colab.research.google.com/github/imhyunho99/2023-1--Deaplearning_Framework/blob/main/6_4_4_3_Char_RNN_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Character Recurrent Neural Network
- mimicing Shakespeare's writing style
- LSTM

In [ ]:
!rm -r data
import os 

try:
  os.mkdir("./data")
except:
  pass

!wget https://raw.githubusercontent.com/dmlc/web-data/master/mxnet/tinyshakespeare/input.txt -P ./data

rm: cannot remove 'data': No such file or directory
--2023-05-08 05:12:35--  https://raw.githubusercontent.com/dmlc/web-data/master/mxnet/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘./data/input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2023-05-08 05:12:35 (19.0 MB/s) - ‘./data/input.txt’ saved [1115394/1115394]



## 1. Settings
### 1) Import required libraries

In [ ]:
import torch
import torch.nn as nn

In [ ]:
!pip install unidecode
import unidecode
import string
import random
import re
import time, math

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 7.0 MB/s eta 0:00:00


## 2) Hyperparameter

In [ ]:
num_epochs = 10000
print_every = 100
plot_every = 10
chunk_len = 50
hidden_size = 100
batch_size = 1
num_layers = 1
embedding_size = 70
lr = 0.002

## 2. Data
### 1) Prepare characters

In [ ]:
all_characters = string.printable
n_characters = len(all_characters)
print(all_characters)
print('num_chars = ', n_characters)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

num_chars =  100


### 2) Get text data

In [ ]:
f = open('/content/PRACTICE.txt','r')
file = f.readline()
print(file)
file_len = len(file)
print('file_len =', file_len)

Hello my name is hyunho. I'm 25 years old. I'm studying about computer science. I like tennis. So usally I play tennis. And also I like wiskey and wine. I sometimes visit nice wiskey bar. If you are interested in tennis and wiskey, Let's do it with me.
file_len = 252


## 3. Functions for text processing
### 1) Random Chunk

In [ ]:
def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

print(random_chunk())

ey and wine. I sometimes visit nice wiskey bar. If 


### 2) Character to tensor

In [ ]:
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return tensor

print(char_tensor('ABCdef'))

tensor([36, 37, 38, 13, 14, 15])


### 3) Chunk into input & label

In [ ]:
def random_training_set():    
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

## 3. Model & Optimizer
### 1) Model

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        
        self.encoder = nn.Embedding(self.input_size, self.embedding_size)
        self.rnn = nn.LSTM(self.embedding_size,self.hidden_size,self.num_layers)
        self.decoder = nn.Linear(self.hidden_size, self.output_size)
        
    
    def forward(self, input, hidden, cell):
        out = self.encoder(input.view(1,-1))
        out,(hidden,cell) = self.rnn(out,(hidden,cell))
        out = self.decoder(out.view(batch_size,-1))
        return out,hidden,cell

    def init_hidden(self):
        hidden = torch.zeros(self.num_layers,batch_size,self.hidden_size)
        cell = torch.zeros(self.num_layers,batch_size,self.hidden_size)
        return hidden,cell
    

model = RNN(n_characters, embedding_size, hidden_size, n_characters, num_layers)

In [ ]:
inp = char_tensor("A")
print(inp)
hidden,cell = model.init_hidden()
print(hidden.size())

out,hidden,cell = model(inp,hidden,cell)
print(out.size())

tensor([36])
torch.Size([1, 1, 100])
torch.Size([1, 100])


### 2) Loss & Optimizer

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss()

### 3) Test function

In [ ]:
def test():
    start_str = "b"
    inp = char_tensor(start_str)
    hidden,cell = model.init_hidden()
    x = inp

    print(start_str,end="")
    for i in range(200):
        output,hidden,cell = model(x,hidden,cell)

        output_dist = output.data.view(-1).div(0.8).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        predicted_char = all_characters[top_i]

        print(predicted_char,end="")

        x = char_tensor(predicted_char)

## 4. Train

In [ ]:
for i in range(num_epochs):
    inp,label = random_training_set()
    hidden,cell = model.init_hidden()

    loss = torch.tensor([0]).type(torch.FloatTensor)
    optimizer.zero_grad()
    for j in range(chunk_len-1):
        x  = inp[j]
        y_ = label[j].unsqueeze(0).type(torch.LongTensor)
        y,hidden,cell = model(x,hidden,cell)
        loss += loss_func(y,y_)

    loss.backward()
    optimizer.step()
    
    if i % 10 == 0:
        print("\n",loss/chunk_len,"\n")
        test()
        print("\n\n")

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.



 tensor([0.0677], grad_fn=<DivBackward0>) 

bout computer science. I like tennis. So usally I play tennis. And also I like wiskey and wine. I sometimes visit nice wiskey bar. If you are interested in tennis and wiskey, Let's do it wikey tennis. 



 tensor([0.0774], grad_fn=<DivBackward0>) 

bar. If you are interested in tennis and wiskey, Let's do it with mplay I sometimes visit nice wiskey bar. If you are interested in tennis. So usally I play tennis. And also I like wiskey and wine. I s



 tensor([0.0707], grad_fn=<DivBackward0>) 

bout computer science. I like tennis. And also I like wiskey and wine. I sometimes visit nice wiskey bar. If you are interested in tennis and wiskey, Let's do it with me inter science. I like tennis. S



 tensor([0.1120], grad_fn=<DivBackward0>) 

b. I'm 25 years old. I'm studying about computer science. I like tennis. So usally I play tennis. And also I like wiskey and wine. I sometimes visit nice wiskey bar. If y

In [ ]:
PRACTICE = "Hello my name is hyunho. I'm 25 years old. I'm studying about computer science. I like tennis. So usally I play tennis. And also I like wiskey and wine. I sometimes visit nice wiskey bar. If you are interested in tennis and wiskey, Let's do it with me."

In [ ]:
f = open("/content/PRACTICE.txt", 'w')
f.write(PRACTICE)

252